[Reference](https://github.com/microsoft/hummingbird)

In [1]:
!pip install hummingbird_ml

     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 7.1MB 14.2MB/s 
     |████████████████████████████████| 7.4MB 53.5MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [2]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from hummingbird.ml import convert

In [3]:
# We are going to use the breast cancer dataset from scikit-learn for this example.
X, y = load_breast_cancer(return_X_y=True)
nrows=15000
X = X[0:nrows]
y = y[0:nrows]

In [4]:
# Create and train a random forest model.
model = RandomForestClassifier(n_estimators=10, max_depth=10)
model.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [5]:
%%timeit -r 3

# Time for scikit-learn.
model.predict(X)

1000 loops, best of 3: 1.33 ms per loop


In [6]:
model = convert(model, 'torch', extra_config={"tree_implementation":"gemm"})

In [7]:
%%timeit -r 3

# Time for HB.
model.predict(X)

The slowest run took 131.75 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.51 ms per loop


In [8]:
model.to('cuda')

PyTorchBackendModelClassification(
  (operator_map): ModuleDict(
    (SklearnRandomForestClassifier): GEMMDecisionTreeImpl()
  )
)

In [9]:
%%timeit -r 3

# Time for HB GPU.
model.predict(X)

The slowest run took 150.06 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 404 µs per loop
